In [1]:
import mysql.connector

In [4]:
conn = mysql.connector.connect(user = 'u690159757_kim', 
                          password = '*yKa&T6y4#sJ8XvE9j',
                          database = 'u690159757_kim',
                          host = '109.106.254.1',
                          connection_timeout = 5 
                          )

In [5]:
cursor = conn.cursor()
cursor.execute(''' SELECT * FROM movies''')
data = cursor.fetchall()
conn.commit()
cursor.close()

movies = [{'movie_id':row[0],'title': row[1]} for row in data]


In [6]:
movies

[{'movie_id': 1, 'title': 'Creed III'},
 {'movie_id': 2, 'title': 'Avatar: The Way of Water'},
 {'movie_id': 3, 'title': 'The Super Mario Bros. Movie'},
 {'movie_id': 4, 'title': 'Mummies'},
 {'movie_id': 5, 'title': 'Supercell'}]